In [1]:
library(magrittr)
library(caret)
library(cattonum)
library(data.table)
taxi=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\taxi_data1.csv',sep=',')
str(taxi)
taxi.y=taxi$fare_amount

##decompose time
year=month=day=hour=min=sec=weekday=week=rep(0,nrow(taxi))
for (i in 1:nrow(taxi)) {
  t=as.character(taxi$pickup_datetime[i]) %>% strsplit(.," ") %>% unlist(.)
  time.1=t[1]
  time.2=t[2]
  
  t1=time.1%>%strsplit(.,"-")%>% unlist(.) %>% as.numeric()
  t2=time.2%>%strsplit(.,":")%>% unlist(.)%>% as.numeric()
  weekday[i]=weekdays(as.Date(time.1))
  week[i]=week(as.Date(time.1))
  
  year[i]=t1[1]
  month[i]=t1[2]
  day[i]=t1[3]
  
  hour[i]=t2[1]
  min[i]=t2[2]
  sec[i]=t2[3]
}

taxi.new=cbind(taxi[,-2],year,month,day,hour,min,sec)

## regression
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
reg.cv= train(fare_amount~., data=taxi.new, trControl=train_control, method="lm")
t12=Sys.time()
time1=difftime(t11,t12,units='secs')
rmse1=reg.cv$results[2]
rsq1=reg.cv$results[3]

##xgbppst
t11=Sys.time()
xgb1=train(fare_amount~., data=taxi.new, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.req1=mean(xgb1$results$Rsquared)
gb.rmse1=mean(xgb1$results$RMSE)
t12=Sys.time()

time.gb1=difftime(t11,t12,units='secs')

###regression+week+weekday
weekday=as.numeric(factor(weekday,levels = c('星期一',
                                             '星期二',
                                             '星期三',
                                             '星期四',
                                             '星期五',
                                             '星期六',
                                             '星期日')))

taxi.new.new=cbind(taxi.new,weekday,week)
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
reg.cv.2= train(fare_amount~., data=taxi.new.new, trControl=train_control, method="lm")
t12=Sys.time()
time2=difftime(t11,t12,units='secs')
rmse2=reg.cv.2$results[2]
rsq2=reg.cv.2$results[3]

###xgboost+week+weekday
t11=Sys.time()
xgb2=train(fare_amount~., data=taxi.new.new, trControl=train_control, method="xgbTree")
gb.req2=mean(xgb2$results$Rsquared)
gb.rmse2=mean(xgb2$results$RMSE)
t12=Sys.time()

time.gb2=difftime(t11,t12,units='secs')


result1=data.frame(type=c('reg','reg+week','xgb','xgb+week'),
                   rsq=as.numeric(c(rsq1,rsq2,gb.req1,gb.req2)),
                   rmse=as.numeric(c(rmse1,rmse2,gb.rmse1,gb.rmse2)),
                   time=-c(time1,time2,time.gb1,time.gb2))
result1




Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: ggplot2

Attaching package: 'xgboost'

The following object is masked from 'package:dplyr':

    slice



ERROR: Error in library(dummies): there is no package called 'dummies'


###  adding week and weekdays can slightly improve model performance of xgboost. However, in regression, adding week and weekdays results in worse performance.

In [1]:
#年週期與周週期
year.cycle=cos((month/6 + day/180 )*pi)
week.cycle=sin((weekday/3.5 + hour/84 )*pi)
taxi.new.new.new=cbind(taxi.new.new,week.cycle,year.cycle)

###regression+cycle
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
reg.cv3= train(fare_amount~., data=taxi.new.new.new, trControl=train_control, method="lm")
t12=Sys.time()
time3=difftime(t11,t12,units='secs')
rmse3=reg.cv3$results[2]
rsq3=reg.cv3$results[3]

### xgboost+cycle
t11=Sys.time()
xgb3=train(fare_amount~., data=taxi.new.new.new, trControl=train_control, method="xgbTree")
gb.req3=mean(xgb3$results$Rsquared)
gb.rmse3=mean(xgb3$results$RMSE)
t12=Sys.time()

time.gb3=difftime(t11,t12,units='secs')

result2=data.frame(type=c('reg','reg+week','reg+week+cycle','xgb','xgb+week','xgb+week+cycle'),
                   rsq=as.numeric(c(rsq1,rsq2,rsq3,gb.req1,gb.req2,gb.req3)),
                   rmse=as.numeric(c(rmse1,rmse2,rmse3,gb.rmse1,gb.rmse2,gb.rmse3)),
                   time=-c(time1,time2,time3,time.gb1,time.gb2,time.gb3))
result2


ERROR: Error in eval(expr, envir, enclos): 找不到物件 'month'


###  adding cycle cannot improve model performance of regression and xgboost.